In [43]:
import xarray as xr
import pandas as pd 
import numpy as np

## Reproduce the bug

Example taken from [DataArray.resample](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.resample.html), adapted to name the dimension `valid_time`

In [44]:
xda = xr.DataArray(
    np.linspace(0, 11, num=12),
    coords={"valid_time":
        pd.date_range(
            "1999-12-15",
            periods=12,
            freq=pd.DateOffset(months=1),
        )
    },
)
print(xda)

<xarray.DataArray (valid_time: 12)>
array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 1999-12-15 2000-01-15 ... 2000-11-15


In [45]:
try:
    data_per_day = xda.resample(valid_time="1D").interpolate("polynomial")
except ValueError as err:
    assert str(err) == "order is required when method=polynomial"

In [46]:
try:
    data_per_day = xda.resample(valid_time="1D").interpolate(kind="polynomial", order=3)
except TypeError as err:
    assert (
        str(err) == "Resample.interpolate() got an unexpected keyword argument 'order'"
    )

## Technical Analysis

First error message

> "order is required when method=polynomial"

Source: https://github.com/pydata/xarray/blob/main/xarray/core/missing.py#L153

It seems that the `method` argument becomes the `order` one when `method == 'polynomial'`. It is unclear to me what the `order` argument is supposed to contain. Maybe @jhamman have an idea? This seems to have been introduced in https://github.com/pydata/xarray/pull/1640

Second error message:

> "Resample.interpolate() got an unexpected keyword argument 'order'"

Check https://docs.xarray.dev/en/stable/generated/xarray.core.resample.DataArrayResample.interpolate.html

Source: https://github.com/pydata/xarray/blob/main/xarray/core/resample.py#L143-L171

We can see that the function signature only allows for a single `kind` keyword, hence the `TypeError: Resample.interpolate() got an unexpected keyword argument 'order'` is the direct consequence.

It uses [interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html). :warning: This function is declared as legacy, so maybe xarray should move away from its use?


## `xr.show_versions()`

In [47]:
import warnings

warnings.filterwarnings("ignore")
xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.10.12 (main, Aug 15 2023, 11:50:32) [GCC 9.4.0]
python-bits: 64
OS: Linux
OS-release: 5.15.0-92-generic
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: ('en_US', 'UTF-8')
libhdf5: None
libnetcdf: None

xarray: 2023.10.1
pandas: 2.1.3
numpy: 1.25.2
scipy: 1.12.0
netCDF4: None
pydap: None
h5netcdf: None
h5py: None
Nio: None
zarr: 2.16.1
cftime: None
nc_time_axis: None
PseudoNetCDF: None
iris: None
bottleneck: None
dask: 2023.11.0
distributed: None
matplotlib: None
cartopy: None
seaborn: None
numbagg: None
fsspec: 2023.10.0
cupy: None
pint: None
sparse: None
flox: None
numpy_groupies: None
setuptools: 67.8.0
pip: 23.1.2
conda: None
pytest: None
mypy: None
IPython: 8.17.2
sphinx: None
